# Course 2: Advanced Portfolio Construction and Analysis With Python
## Module 1: Style & Factors

**Question 1**

Load the 49 Value Weighted industry portfolio returns. Limit the analysis to returns of the 49 VW portfolios from 1991 onwards (1991 included, 1990 not included, 2018 included). Also load the Fama French Research Factors over the same period. Use the Mkt-RF as in the lab notebooks to compute the CAPM betas.

What is the CAPM (Single Factor) Beta when evaluated over the entire period (1991-2018) of Beer?

Enter the answer as a number to at least two decimal places (e.g. 0.92)

In [1]:
import pandas as pd

#Load the 49 Value Weighted industry portfolio returns from 1991 onwards
ind_returns = pd.read_csv("data/ind49_m_vw_rets.csv", header=0, index_col = 0, na_values=-99.99)
ind_returns.index = pd.to_datetime(ind_returns.index, format = '%Y%m').to_period('M')
ind_returns = ind_returns.loc['1991':,:]/100
ind_returns.columns = ind_returns.columns.str.strip()

#load the Fama French Research Factors over the same period
fff = pd.read_csv("data/F-F_Research_Data_Factors_m.csv",header=0, index_col=0, na_values=-99.99)
fff.index = pd.to_datetime(fff.index, format="%Y%m").to_period('M')
fff = fff.loc['1991':, :]/100
fff.head()

,Mkt-RF,SMB,HML,RF
1991-01,0.0469,0.0379,-0.0184,0.0052
1991-02,0.0719,0.0395,-0.0054,0.0048
1991-03,0.0265,0.0389,-0.0123,0.0044
1991-04,-0.0028,0.0050,0.0142,0.0053
1991-05,0.0365,-0.0034,-0.0057,0.0047


Next, we need to decompose the observed portfolio return (Beer) into the portion that's due to the market and the rest that is not due to the market, using the CAPM as the explanatory model.

i.e.: $$ R_{portfolio,t} - R_{f,t} = \alpha + \beta(R_{mkt,t} - R_{f,t}) + \epsilon_t $$

We can use the stats.api for the linear regression as follows:

In [2]:
import statsmodels.api as sm
import numpy as np

beer_excess = ind_returns["Beer"] - fff["RF"]
fff["Constant"] = 1
exp_var = fff[["Mkt-RF","Constant"]]
lm = sm.OLS(beer_excess, exp_var).fit()
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.226
Model:                            OLS   Adj. R-squared:                  0.224
Method:                 Least Squares   F-statistic:                     97.55
Date:                Wed, 23 Sep 2020   Prob (F-statistic):           2.37e-20
Time:                        22:52:40   Log-Likelihood:                 596.94
No. Observations:                 336   AIC:                            -1190.
Df Residuals:                     334   BIC:                            -1182.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         0.5295      0.054      9.877      0.000       0.424       0.635
Constant       0.0043      0.002      1.892      0.059      -0.000       0.009
==============================================================================
Omnibus:                       20.697   Durbin-Watson:                   2.079
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               43.130
Skew:                          -0.314   Prob(JB):                     4.31e-10
Kurtosis:                       4.639   Cond. No.                         23.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [3]:
#beta
lm.params[0].round(2)

0.53

**Question 2**

Using the same data as the previous question, what is the CAPM Beta when evaluated over the entire period (1991-2018) of Steel?

Enter your answer as a number to at least 2 decimal places (e.g. 1.12)

In [4]:
steel_excess = ind_returns["Steel"] - fff["RF"]
exp_var = fff[["Mkt-RF","Constant"]]
lm = sm.OLS(steel_excess, exp_var).fit()
lm.params[0].round(2)

1.55

**Question 3**

Using the same data as the previous question, what is the CAPM Beta when evaluated over the 2013-2018 (both included) period of Beer?

In [5]:
beer_excess = ind_returns.loc["2013":,"Beer"] - fff.loc["2013":,"RF"]
exp_var = fff.loc["2013":,["Mkt-RF","Constant"]]
lm = sm.OLS(beer_excess, exp_var).fit()
lm.params[0].round(2)

0.59

**Question 4**

Using the same data set as the previous question, what is the CAPM Beta when evaluated over the 2013-2018 (both included) period of Steel?

In [6]:
steel_excess = ind_returns.loc["2013":,"Steel"] - fff.loc["2013":,"RF"]
exp_var = fff.loc["2013":,["Mkt-RF","Constant"]]
lm = sm.OLS(steel_excess, exp_var).fit()
lm.params[0].round(2)

1.42

**Question 5**

Using the same data as the previous question, which of the 49 industries had the highest CAPM Beta when evaluated over the 1991-1993 (both included) period? (Use the same industry names as in the files). Enter the name as a text string, and remember to exactly match the column headers in the data file.

In [7]:
ind_excess = ind_returns.loc["1991":"1993"]- fff.loc["1991":"1993",["RF"]].values
exp_var = fff.loc["1991":"1993",["Mkt-RF","Constant"]]
betas = [sm.OLS(ind_excess[col], exp_var).fit().params[0].round(2) for col in ind_excess.columns]
betas_df = pd.DataFrame(betas, index = ind_excess.columns)
betas_df.head()

,0
Agric,1.03
Food,1.26
Soda,1.32
Beer,1.20
Smoke,1.32


In [8]:
betas_df.idxmax()

0    Hlth
dtype: object

**Question 6**

Using the same data as the previous question, which of the 49 industries had the lowest CAPM Beta when evaluated over the 1991-1993 (both included) period? (Use the same industry names as in the files) . Enter the answer as text and remember to exactly match the column headers in the data file.

In [9]:
betas_df.idxmin()

0    Gold
dtype: object

**Question 7**

For the next 4 questions use the Full 3 Factor Fama-French model using the research data supplied in the following data file:

F-F_Research_Data_Factors.csv

and the same 1991-2018 period you have just used for the previous questions.

Of the 49 industries, which displayed the highest Small Cap tilt when analyzed over the entire 1991-2018 period?

Enter your answer as a text string, and remember to use the exact same name as the column header in the text file

In [10]:
fff.head()

,Mkt-RF,SMB,HML,RF,Constant
1991-01,0.0469,0.0379,-0.0184,0.0052,1
1991-02,0.0719,0.0395,-0.0054,0.0048,1
1991-03,0.0265,0.0389,-0.0123,0.0044,1
1991-04,-0.0028,0.0050,0.0142,0.0053,1
1991-05,0.0365,-0.0034,-0.0057,0.0047,1


SMB: Small Minus Big - Long small cap stocks and short large cap stocks

HML: High Minus Low - Long high value stocks and short growth stocks

In [11]:
def regress(dependent_variable, explanatory_variables, alpha=True):
    """
    Runs a linear regression to decompose the dependent variable into the explanatory variables
    returns an object of type statsmodel's RegressionResults on which you can call
       .summary() to print a full summary
       .params for the coefficients
       .tvalues and .pvalues for the significance levels
       .rsquared_adj and .rsquared for quality of fit
    """
    if alpha:
        explanatory_variables = explanatory_variables.copy()
        explanatory_variables["Alpha"] = 1

    lm = sm.OLS(dependent_variable, explanatory_variables).fit()
    return lm

ind_excess = ind_returns - fff[["RF"]].values
fff = fff.drop("RF", axis=1)

In [12]:
betas_df = regress(ind_excess, fff, alpha = False).params
betas_df.columns = ind_excess.columns
betas_df.T.head()

,Mkt-RF,SMB,HML,Constant
Agric,0.684404,0.165856,0.131897,0.002848
Food,0.558117,-0.267873,0.156536,0.002489
Soda,0.817167,-0.121587,0.397642,0.003340
Beer,0.590261,-0.358655,0.006329,0.004493
Smoke,0.590791,-0.307794,0.266018,0.004717


In [13]:
betas_df.loc["SMB"].idxmax()

'FabPr'

**Question 8**

Using the same dataset and period as the previous question, of the 49 industries, which displayed the highest Large Cap tilt when analyzed over the entire period?

In [14]:
betas_df.loc["SMB"].idxmin()

'Beer'

**Question 9**

Using the same data as period as the previous question, of the 49 industries, which displayed the highest Value tilt when analyzed over the entire period?

In [15]:
betas_df.loc["HML"].idxmax()

'Txtls'

**Question 10**

Using the same data set and period as the previous question, of the 49 industries, which displayed the highest Growth tilt when analyzed over the entire period?

In [16]:
betas_df.loc["HML"].idxmin()

'Softw'